# Example of generating a GVEC equilibrium

This example demonstrates how to:
- Create a GVEC equilibrium configuration using the GVEC library
- Visualize the geometry and profiles
- Export to GYSELA format (HDF5)

**Note**: This requires the `gvec` package to be installed. See https://github.com/gvec/gvec for installation instructions.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gysmc import GvecMagnetConfig, QProfile, GYSMagnetConfig

# Check if GVEC is available
try:
    import gvec
    print("GVEC is available!")
except ImportError:
    print("WARNING: GVEC is not available. Please install it using:")
    print("  pip install gvec")
    print("or see https://github.com/gvec/gvec for installation instructions")


: 

### Set up the GVEC equilibrium parameters ###

Two options are available:

a) Read from parameter file (VMEC or Gvec format)

b) pass parameters same as culham equilibria:

- **major_radius**: Major radius at magnetic axis (R0)
- **q_profile**: Safety factor profile (QProfile object or None for default)
- **pressure_profile**: Pressure profile (optional, can be array or object with get_pressure method)
- **rmax**: Maximum radial coordinate (should match GYSELA rmax)
- **beta_toro**: Toroidal beta parameter
- **kappa_elongation**: Elongation at edge
- **delta_triangularity**: Triangularity at edge
- **runpath**: Optional path for GVEC run directory (uses temporary directory if None)

In [ ]:
# Create a parabolic q profile: q(r) = q0 + qa * (r/a)^2
# Option 2: Parabolic profile
q_profile = QProfile(option=2, q_param1=1.0, q_param2=2.7858, q_param3=2.8, rmax=1.2)

# Initialize GVEC equilibrium
# Note: This will run GVEC equilibrium solver, which may take a few minutes
gvecmc = GvecMagnetConfig(
    major_radius=3.0,         # R0 = 3.0
    q_profile=q_profile,      # Safety factor profile
    pressure_profile=None,    # No pressure profile (beta=0)
    rmax=1.2,                 # Maximum radial coordinate
    beta_toro=0.0,            # Toroidal beta
    kappa_elongation=1.8,      # Elongation kappa = 1.8
    delta_triangularity=0.4,  # Triangularity delta = 0.4
    runpath=None              # Use temporary directory
)

print("GVEC equilibrium initialized successfully!")


### Visualize the equilibrium ###

We can plot the q profile and geometry to verify the equilibrium.

In [ ]:
# Plot q profile
r_array = np.linspace(0.01, 1.2, 200)
q_values = gvecmc.get_q(r_array)

plt.figure(figsize=(8, 6))
plt.plot(r_array, q_values, 'b-', linewidth=2)
plt.xlabel('r/a', fontsize=12)
plt.ylabel('q(r)', fontsize=12)
plt.title('Safety Factor Profile (GVEC)', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xlim(0, 1.2)
plt.show()


: 

In [ ]:
# Plot geometry (flux surfaces)
r_plot = np.linspace(0.1, 1.2, 16)
theta_plot = np.linspace(0, 2*np.pi, 64)
phi_plot = np.array([0.0])

R, Z = gvecmc.get_RZ(r_plot, theta_plot, phi_plot)

plt.figure(figsize=(10, 10))
for i in range(len(r_plot)):
    plt.plot(R[i, :, 0], Z[i, :, 0], 'b-', linewidth=1.5, alpha=0.7)

# Plot the last surface (edge) in red
plt.plot(R[-1, :, 0], Z[-1, :, 0], 'r-', linewidth=2)

plt.axis('equal')
plt.xlabel('R (normalized)', fontsize=12)
plt.ylabel('Z (normalized)', fontsize=12)
plt.title('GVEC Equilibrium: Flux Surfaces', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()


### Write to GYSELA magnet_config.h5 ###

```Nr```, ```Ntheta```, ```minor_radius```, ```rmin```, ```rmax```, ```skiphole``` should match that of a GYSELA input file


In [ ]:
# Convert to GYSELA format and save
gysconfig = GYSMagnetConfig(
    gvecmc,
    Nr=127,              # Number of radial points (should match GYSELA)
    Ntheta=256,          # Number of poloidal points (should match GYSELA)
    minor_radius=100,    # Minor radius in cm (should match GYSELA)
    rmax=1.2,            # Maximum radial coordinate (should match GYSELA)
    rmin=0.0,            # Minimum radial coordinate
    skiphole=True        # Skip hole in mesh
)

# Plot geometry
gysconfig.plot_geometry(N_surf=16, N_theta_plot=32)
plt.show()

# Save to HDF5
gysconfig.to_hdf5('gvec_equilibrium.h5')
print("Saved to gvec_equilibrium.h5")


### How to use the generated hdf5 ###

- Copy it to ```gysela``` folder under ```wk/profiles```.

- Set ```magnet_strategy = "EXTERNAL"``` in your ```EQUIL``` namelist.

- Specify the path to the hdf5 file such as ```magnet_config_filename = profiles/magnet_config.h5``` in the ```Variables for radial profile input files``` section.


### Example with pressure profile ###

You can also include a pressure profile for finite beta equilibria:


In [ ]:
# Example: Create a pressure profile
r_pressure = np.linspace(0.0, 1.2, 100)
p_profile = (1.0 - r_pressure**2)**2  # Parabolic pressure profile

# Create equilibrium with pressure
# Note: This will run GVEC again, which may take a few minutes
gvecmc_beta = GVecMagnetConfig(
    major_radius=3.0,
    q_profile=q_profile,
    pressure_profile=p_profile,  # Include pressure profile
    rmax=1.2,
    beta_toro=0.02,              # Beta = 2%
    kappa_elongation=1.8,
    delta_triangularity=0.4,
    runpath=None                  # Use temporary directory
)

print("GVEC equilibrium with pressure initialized!")
